In [12]:
from numpy import exp, array, random, dot
import numpy as np
from scipy import misc
import skimage
from __future__ import with_statement

In [13]:
arr1 = np.divide(misc.imread('tAmerica1.png'),255)
arr1r = arr1[:,:,0]
arr1g = arr1[:,:,1]
arr1b = arr1[:,:,2]
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [39]:
filterNum = 3
filterSize = 10
layerNum = 6

In [40]:
filters = []
for x in range(filterSize*layerNum):
    filters += [0.01*random.random((filterSize,filterSize))-0.005] #run previous before this 
filters = array(filters)
print(filters)

[[[ -2.92515055e-03   4.57473206e-03  -1.86720768e-04 ...,   2.82041433e-03
     1.29667701e-04  -1.59019878e-03]
  [  4.96696928e-04   3.64263916e-03  -3.19318332e-03 ...,   3.29945741e-03
    -4.88249605e-03  -4.47614042e-06]
  [  4.36591634e-03  -2.12976830e-03  -1.57261821e-03 ...,  -3.57568480e-03
     7.16751626e-04   2.98755558e-03]
  ..., 
  [  3.56015290e-03  -3.02517570e-03  -2.67751230e-04 ...,   1.47686660e-03
     2.66606893e-03   4.30678679e-03]
  [ -1.38651072e-03  -1.40799528e-03  -4.39237445e-03 ...,   2.87583157e-03
    -8.53643507e-04  -4.39221085e-03]
  [  4.73061478e-04  -2.70361960e-04  -2.68533692e-03 ...,   2.30693466e-03
    -1.67277405e-03   3.49223086e-03]]

 [[ -4.97733038e-03  -4.35658284e-03  -5.28347257e-04 ...,  -1.56441095e-03
     4.61552235e-03  -2.57803019e-03]
  [ -3.18571776e-03   2.12799916e-03   4.49340484e-03 ...,  -3.92852362e-03
     4.10374384e-03  -1.24308721e-03]
  [  4.70328471e-04  -3.10270360e-03  -3.98184859e-03 ...,   2.56099042e-03
  

In [41]:
def activation(x, deriv=False):
    if deriv:
        if x >= 0:
            return 1
        else:
            return 0
    return max(0,x)
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
def apply(filterArr, img, row, col):
    value = 0
    for y in range(len(filterArr)):
        for x in range(len(filterArr[0])):
            value += img[y+row][x+col]*filterArr[y][x]
    return value/sumNoNeg(filterArr)
print(apply(filters[4],arr1r,0,0))

-0.0369663189141


In [42]:
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)-filterSize):
        for x in range(len(img[y])-filterSize):
            new_img[y][x] += activation(apply(filterArr,img,y,x))
    return new_img
print(len(([[0]*(len(arr1r)-filterSize)]*(len(arr1r[0])-filterSize))[0]))
print(len(arr1r[0])-filterSize)
#print([[0]*(len(arr1r)-filterSize)]*(len(arr1r[0])-filterSize))
print(applyFilters(arr1r, np.zeros((len(arr1r)-filterSize, len(arr1r[0])-filterSize)), filters[0]))


148
290
[[ 0.03771454  0.0225282   0.05135443 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.00182292  0.01852871  0.0631659  ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.          0.00419298  0.         ...,  0.03753356  0.03753356
   0.03753356]
 ..., 
 [ 0.05235905  0.05235905  0.05235905 ...,  0.05235905  0.05235905
   0.05235905]
 [ 0.03753356  0.03753356  0.03753356 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.03753356  0.03753356  0.03753356 ...,  0.03753356  0.03753356
   0.03753356]]


In [43]:
def forward(img):
    outputLayers = [img]
    for num in range(layerNum):
        newImage = np.zeros((len(outputLayers[num])-filterSize, len(outputLayers[num][0])-filterSize))
        for fil in range(filterNum):
            applyFilters(outputLayers[num], newImage, filters[filterNum*num + fil])
            print(filterNum*num + fil)
            print(newImage)
        outputLayers.append(np.divide(newImage, filterNum))
    return outputLayers
out = forward(arr1r)
print(out)

0
[[ 0.03771454  0.0225282   0.05135443 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.00182292  0.01852871  0.0631659  ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.          0.00419298  0.         ...,  0.03753356  0.03753356
   0.03753356]
 ..., 
 [ 0.05235905  0.05235905  0.05235905 ...,  0.05235905  0.05235905
   0.05235905]
 [ 0.03753356  0.03753356  0.03753356 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.03753356  0.03753356  0.03753356 ...,  0.03753356  0.03753356
   0.03753356]]
1
[[ 0.04889799  0.05408993  0.09156125 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.01518844  0.02501973  0.11374217 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.          0.00419298  0.00226235 ...,  0.03753356  0.03753356
   0.03753356]
 ..., 
 [ 0.05235905  0.05235905  0.05235905 ...,  0.05235905  0.05235905
   0.05235905]
 [ 0.03753356  0.03753356  0.03753356 ...,  0.03753356  0.03753356
   0.03753356]
 [ 0.03753356  0.03753356  0.03753356 ...,  0.03753356  0.03753356
   0.0375335

In [19]:
text_file = open("text.txt", "w")
text_file.write(np.array_str(array(out)))
text_file.close()